## Export models for use in production

So, we build a great model in TensorFlow and we want to deploy it into a production environment. What should we do? Well, one of the best features of TensorFlow is that we can take a model we've built, like this one, and export it to a file and then run that file on Google's cloud servers. That let's us scale up any machine learning feature we've built to an almost infinite scale without having to maintain our own servers. But in order to do that we have to tell Google how we want to run our model in production.

Let's take a closer look at this model in TensorBoard. There's several ways we can use this model. First, if we want to initialize all the variables to their default values, we call this init operator. If we want to generate an output we can pass an input data and then call the output operation. And if we want to train the network we can call the train operator. If we export this model to a file using the normal way of saving model checkpoint files, Google won't know which function we want to run. Instead, we need to export this model a special way where define exactly what the start in point of the model is that we want to run. 
Let's take a look. 

![model](images/trained_model.png)

Here we've defined our model with a normal computational graph and a training loop. At the end of the training loop we want to export the train model, but because we want to use the model in the cloud, we can't use the normal tf.train.saver function to save the model. Instead, we're going to use TensorFlow's saved model builder that let's us customize how the model is exported. 

First, let's create a new saved model builder object. The full name is tf.saved_model.builder.SavedModelBuilder. The only parameter we have to pass in is the name of the folder we want to save the model file to. We are going to use exported model. Now we're ready to define the inputs and outputs of the model that we want Google to use when it runs the model in the cloud.

***Input Data**
First, we have a Python dictionary with a key called inputs. In this dictionary, we'll list each tensor that needs to be filled in when their model is run. Our model takes in one tensor with nine values as input, so that means our model will have one input. We'll just call it input. For the value of the input key, we'll pass in the tensor we want data fed into. In this case, it will be X, but to make this work we also need to wrap that tensor in the call to a function called tf.saved_model.utils.build_tensor_info, like this, and then pass in X. Okay, now let's define the output the same way. Our output is a single tensor with one value. Let's call the output earnings and pass in the prediction tensor as the tensor to output. Again we'll make the call to tf.saved_model.utils.build_tensor_info and pass in prediction. Next, on line 158, we have to define what TensorFlow calls a signature def. A signature def is sort of like a function or method declaration in the programming language. We're telling TensorFlow that to run the model it should call a certain function with certain parameters. We create the signature def by calling this tf.saved_model.signature_def_utils.build_signature_def then we'll pass in the inputs and outputs we just defined. So the inputs will be the inputs dictionary and the outputs will be the outputs dictionary. Then we have to name the method we are defining, but we won't make up a name, we'll always use the special predefined function name called tf.saved_model.signature_constants.PREDICT_METHOD_NAME. That's the name Google will always look for in order to execute our model. Great, we've defined the inputs, the outputs, and the signature def. Now we're ready to configure the model builder to tell it exactly how we want this model exported. Let's go down to line 164 and then we can call the model_builder.add_meta_graph_and_variables function. That names a mouthful, but meta graph is the structure of our computational graph and the variables are the values we set on each node in the graph. So this is telling TensorFlow that we want to export everything. Next, it takes in the session and then the tag name which is what Google looks for when figuring out what to execute. So it'll always be tf.saved_model.tag_constants.SERVING. Next, we create a signature def map that lists all the signature defs this model supports. This map has an entry with a magic name called tf.saved_model.signature_constants. DEFAULT_SERVING_SIGNATURE_DEF_KEY. That long name is what Google will be looking for. That's it, all that's left to do is call save on our model builder. We'll do that on line 172 and now we can run the code. Right-click and choose run. We can see here in PyCharm a new exported model folder and inside is a file called saved_model.pb. This file contains the structure of our model in Google's special proto buff format. There's also a variables subfolder that contains a checkpoint of all the variables in our graph. This model is now ready to be uploaded to the Google cloud.

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

tf.reset_default_graph() 


# Turn off TensorFlow warning messages in program output
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Load training data set from CSV file
training_data_df = pd.read_csv("sales_data_training.csv", dtype=float)

# Pull out columns for X (data to train with) and Y (value to predict)
X_training = training_data_df.drop('total_earnings', axis=1).values
Y_training = training_data_df[['total_earnings']].values

# Load testing data set from CSV file
test_data_df = pd.read_csv("sales_data_test.csv", dtype=float)

# Pull out columns for X (data to train with) and Y (value to predict)
X_testing = test_data_df.drop('total_earnings', axis=1).values
Y_testing = test_data_df[['total_earnings']].values

# All data needs to be scaled to a small range like 0 to 1 for the neural
# network to work well. Create scalers for the inputs and outputs.
X_scaler = MinMaxScaler(feature_range=(0, 1))
Y_scaler = MinMaxScaler(feature_range=(0, 1))

# Scale both the training inputs and outputs
X_scaled_training = X_scaler.fit_transform(X_training)
Y_scaled_training = Y_scaler.fit_transform(Y_training)

# It's very important that the training and test data are scaled with the same scaler.
X_scaled_testing = X_scaler.transform(X_testing)
Y_scaled_testing = Y_scaler.transform(Y_testing)

# Define model parameters
learning_rate = 0.001
training_epochs = 100
display_step = 5

# Define how many inputs and outputs are in our neural network
number_of_inputs = 9
number_of_outputs = 1

# Define how many neurons we want in each layer of our neural network
layer_1_nodes = 50
layer_2_nodes = 100
layer_3_nodes = 50

# Section One: Define the layers of the neural network itself

# Input Layer
with tf.variable_scope('input'):
    X = tf.placeholder(tf.float32, shape=(None, number_of_inputs))

# Layer 1
with tf.variable_scope('layer_1'):
    weights = tf.get_variable("weights1", shape=[number_of_inputs, layer_1_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases1", shape=[layer_1_nodes], initializer=tf.zeros_initializer())
    layer_1_output = tf.nn.relu(tf.matmul(X, weights) + biases)

# Layer 2
with tf.variable_scope('layer_2'):
    weights = tf.get_variable("weights2", shape=[layer_1_nodes, layer_2_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases2", shape=[layer_2_nodes], initializer=tf.zeros_initializer())
    layer_2_output = tf.nn.relu(tf.matmul(layer_1_output, weights) + biases)

# Layer 3
with tf.variable_scope('layer_3'):
    weights = tf.get_variable("weights3", shape=[layer_2_nodes, layer_3_nodes], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases3", shape=[layer_3_nodes], initializer=tf.zeros_initializer())
    layer_3_output = tf.nn.relu(tf.matmul(layer_2_output, weights) + biases)

# Output Layer
with tf.variable_scope('output'):
    weights = tf.get_variable("weights4", shape=[layer_3_nodes, number_of_outputs], initializer=tf.contrib.layers.xavier_initializer())
    biases = tf.get_variable(name="biases4", shape=[number_of_outputs], initializer=tf.zeros_initializer())
    prediction = tf.matmul(layer_3_output, weights) + biases

# Section Two: Define the cost function of the neural network that will be optimized during training

with tf.variable_scope('cost'):
    Y = tf.placeholder(tf.float32, shape=(None, 1))
    cost = tf.reduce_mean(tf.squared_difference(prediction, Y))

# Section Three: Define the optimizer function that will be run to optimize the neural network

with tf.variable_scope('train'):
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

# Create a summary operation to log the progress of the network
with tf.variable_scope('logging'):
    tf.summary.scalar('current_cost', cost)
    summary = tf.summary.merge_all()

# Initialize a session so that we can run TensorFlow operations
with tf.Session() as session:

    # Run the global variable initializer to initialize all variables and layers of the neural network
    session.run(tf.global_variables_initializer())

    # Create log file writers to record training progress.
    # We'll store training and testing log data separately.
    training_writer = tf.summary.FileWriter('./logs/training', session.graph)
    testing_writer = tf.summary.FileWriter('./logs/testing', session.graph)

    # Run the optimizer over and over to train the network.
    # One epoch is one full run through the training data set.
    for epoch in range(training_epochs):

        # Feed in the training data and do one step of neural network training
        session.run(optimizer, feed_dict={X: X_scaled_training, Y: Y_scaled_training})

        # Every few training steps, log our progress
        if epoch % display_step == 0:
            # Get the current accuracy scores by running the "cost" operation on the training and test data sets
            training_cost, training_summary = session.run([cost, summary], feed_dict={X: X_scaled_training, Y:Y_scaled_training})
            testing_cost, testing_summary = session.run([cost, summary], feed_dict={X: X_scaled_testing, Y:Y_scaled_testing})

            # Write the current training status to the log files (Which we can view with TensorBoard)
            training_writer.add_summary(training_summary, epoch)
            testing_writer.add_summary(testing_summary, epoch)

            # Print the current training status to the screen
            print("Epoch: {} - Training Cost: {}  Testing Cost: {}".format(epoch, training_cost, testing_cost))

    # Training is now complete!

    # Get the final accuracy scores by running the "cost" operation on the training and test data sets
    final_training_cost = session.run(cost, feed_dict={X: X_scaled_training, Y: Y_scaled_training})
    final_testing_cost = session.run(cost, feed_dict={X: X_scaled_testing, Y: Y_scaled_testing})

    print("Final Training cost: {}".format(final_training_cost))
    print("Final Testing cost: {}".format(final_testing_cost))

    # Now that the neural network is trained, let's use it to make predictions for our test data.
    # Pass in the X testing data and run the "prediciton" operation
    Y_predicted_scaled = session.run(prediction, feed_dict={X: X_scaled_testing})

    # Unscale the data back to it's original units (dollars)
    Y_predicted = Y_scaler.inverse_transform(Y_predicted_scaled)

    real_earnings = test_data_df['total_earnings'].values[0]
    predicted_earnings = Y_predicted[0][0]

    print("The actual earnings of Game #1 were ${}".format(real_earnings))
    print("Our neural network predicted earnings of ${}".format(predicted_earnings))
    
#############################################################################################
# Model builder object
#############################################################################################

    model_builder = tf.saved_model.builder.SavedModelBuilder("exported_model")

    inputs = {
        'input': tf.saved_model.utils.build_tensor_info(X)
        }
    outputs = {
        'earnings': tf.saved_model.utils.build_tensor_info(prediction)
        }

    signature_def = tf.saved_model.signature_def_utils.build_signature_def(
        inputs=inputs,
        outputs=outputs,
        method_name=tf.saved_model.signature_constants.PREDICT_METHOD_NAME
    )

    model_builder.add_meta_graph_and_variables(
        session,
        tags=[tf.saved_model.tag_constants.SERVING],
        signature_def_map={
            tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: signature_def
        }
    )

    model_builder.save()
